In [27]:
# Setup
import pandas as pd
from pathlib import Path
data_path = Path('./data/food/')


# 'country_lookup' from '0A - Food System POPULATION inputs 7-5-21.xlsx' workbook:
#  List of countries included in FAO analysis, with various categories used throughout the Food Systems model
countries = pd.read_csv(Path.joinpath(data_path,'countries.txt'), index_col='Country')
old_countries = (countries['CountryExists'] == 0)
old_country_set = set(c.casefold() for c in countries[old_countries].index.to_list())

# 'LCA range' from '0B - Food System LCA inputs 7-5-21.xslx' workbook.
# Farm gate LCA impacts by FAO commodity, with max/min/avg values by study. (g CO2e/kcal food)
LCA_emissions_by_product = pd.read_csv(Path.joinpath(data_path,'LCA_emissions_by_product.txt'), index_col='FAO commodity')
LCA_emissions_by_product.drop('Sort code', axis='columns', inplace=True)

# 'REF_supply' from '0C - Food System FOOD inputs 75 6-19-21.xslx' workbook.
# REF farm gate food supply forecast by country and FAO commodity [kcal/(cap-day)] 
REF_supply = pd.read_csv(Path.joinpath(data_path,'REF_supply.txt'), index_col=['Country', 'FAO commodity'])
idx_countries = set(REF_supply.index.get_level_values(level=0).to_list())
drop_countries = []
for idx_country in idx_countries:
    if idx_country.casefold() in old_country_set:
        drop_countries.append(idx_country)

cols = REF_supply.columns.to_list()
years = [col for col in cols if len(col)==4 and col[:1]=='2']
years_int = [int(y) for y in years]

# 'REF_waste' table from REF_waste tab in '0C - Food System FOOD inputs 75 6-19-21.xslx' workbook.
# REF food waste forecast by country and FAO commodity [kcal/(cap-day)]
REF_waste = pd.read_csv(Path.joinpath(data_path,'REF_waste.txt'), index_col=['Country', 'FAO commodity'])

# 'waste_forecast' table from waste_forecast tab in '0C - Food System FOOD inputs 75 6-19-21.xslx' workbook.
# Mass percentages of food losses and waste (in percentage of what enters each step) (Source: FAO 2011, Global Food Losses.)
waste_forecast = pd.read_csv(Path.joinpath(data_path,'waste_forecast.txt'), index_col=['Waste category', 'Waste region'])


food_lookup  = pd.read_csv(Path.joinpath(data_path,'food_lookup.txt'), index_col=['FAO commodity'])

# From food_lookup tab in "1 - Food System CORE 75 6-18-21.xlsx"
# FAO commodity, Waste category, Diet cateory, Demand category, kcal/g, Yield category:
# List of foods included in the FAO commodity list, with various categories used throughout the Food Systems model.
food_waste_lookup = pd.read_csv(Path.joinpath(data_path,'food_waste_lookup.txt'), index_col=['FAO commodity'])

plant_rich_diet = pd.read_csv(Path.joinpath(data_path,'plant_rich_diet.txt'), index_col=['Country', 'FAO commodity']) # fbs = Food Balance Sheet


# UN Population estimates from World Population Prospects, High Population Scenario, 2014-2100
# Taken from '0A - Food System POPULATION inputs 7-5-21.xlsx', pop_REF1 tab.
population_high = pd.read_csv(Path.joinpath(data_path,'population_high.txt'), index_col='Country')

# UN Population estimates from World Population Prospects, Medium Population Scenario (reflecting "family planning" scenario intervention), 2014-2100
# Taken from '0A - Food System POPULATION inputs 7-5-21.xlsx', pop_REF2 tab.
population_medium = pd.read_csv(Path.joinpath(data_path,'population_medium.txt'), index_col='Country')


parameters ={
    'localization_emisssion_reduction_percentage': 0.05,
    'total_food_waste_reduction': 0.75,
    'diet_start_year': 2019,
    'diet_end_year': 2050,
    'waste_start_year': 2019,
    'waste_end_year': 2050,
    'plant_rich_diet_adoption': 0.75,
    'plant_rich_diet_kcal_per_day': 2300
    }


In [2]:
# Matches waste_forecast table on "Waste_forecast" tab in "1 - Food System CORE 75 6-18-21.xlsx"
waste_forecast['Post-farm gate loss'] =  1-(1-waste_forecast['Postharvest handling and storage'])*(1-waste_forecast['Processing'])*(1-waste_forecast['Packaging']) * \
                                        (1-waste_forecast['Distribution'])*(1-waste_forecast['Consumption'])

# @IF(AND(NUMBERVALUE(waste_forecast[[#Headers],[2014]])>=parameters!$E$6,NUMBERVALUE(waste_forecast[[#Headers],[2014]])<=parameters!$F$6),
#        @waste_forecast[@[Post-farm gate loss]:[Post-farm gate loss]]*(1-parameters!$A$7*(NUMBERVALUE(waste_forecast[[#Headers],[2014]])-parameters!$E$6+1)/(parameters!$F$6-parameters!$E$6+1)),
#       IF(NUMBERVALUE(waste_forecast[[#Headers],[2014]])>parameters!$F$6,@waste_forecast[@[Post-farm gate loss]:[Post-farm gate loss]]*(1-parameters!$A$7),waste_forecast[@[Post-farm gate loss]:[Post-farm gate loss]]))
start_idx = years_int.index(parameters['waste_start_year'])
end_idx = years_int.index(parameters['waste_end_year'])

for y in years_int[:start_idx]:
    waste_forecast[str(y)] = waste_forecast['Post-farm gate loss']

for y in years_int[start_idx:end_idx]:
    waste_forecast[str(y)] = waste_forecast['Post-farm gate loss'] * (1-parameters['total_food_waste_reduction'] * (y - parameters['waste_start_year'] + 1 )/ (parameters['waste_end_year'] - parameters['waste_start_year'] + 1))

for y in years_int[end_idx:]:
    waste_forecast[str(y)] = waste_forecast['Post-farm gate loss'] * (1-parameters['total_food_waste_reduction'])



In [3]:
# Build DIET_supply and DIET_plant_supply to match the two tables on the DIET_supply tab from "1 - Food System CORE 75 6-18-21.xlsx".

# DIET_supply: Per-capita DIET-only total food supply forecast, 2014-2100 [kcal/(cap-day)]
# DIET_plant_supply: DIET food supply forecast *plant-rich diet only*, 2014-2100 [kcal/(cap-day)]

plantrich_demand_2000 = plant_rich_diet.join(other=food_lookup[['Diet category','Waste category']], how='inner').join(countries[['Diet region','Waste region']])[['GrandTotal', 'Diet category', 'Waste category', 'Diet region', 'Waste region']]
plantrich_demand = plantrich_demand_2000.copy()
plantrich_demand['GrandTotal'] = plantrich_demand_2000['GrandTotal'] * parameters['plant_rich_diet_kcal_per_day'] / 2300

arr = []
start_yr = parameters['diet_start_year']
end_yr = parameters['diet_end_year']
adoption = parameters['plant_rich_diet_adoption']
denominator = end_yr - start_yr + 1
for y in range(start_yr, end_yr+1):
    val =  adoption * (y - start_yr +1) / denominator
    arr.append(val)

for y in range(end_yr, years_int[-1]):
    arr.append(adoption)

for y in range(years_int[0],start_yr):
    arr.insert(0,0.0)

REF_demand = REF_supply[years] - REF_waste[years]
DIET_demand = REF_demand.join(food_lookup[['Waste category','Diet category']])
DIET_plant_supply = plantrich_demand.copy()
idx = DIET_plant_supply.index
DIET_plant_supply = pd.merge(DIET_plant_supply, waste_forecast['Post-farm gate loss'], how='left' , on=['Waste category','Waste region'])
DIET_plant_supply.set_index(idx, inplace=True)

for y in years:
    fact = arr.pop(0)
    DIET_demand[y] = REF_demand[y] * (1-fact) + plantrich_demand['GrandTotal'] * fact # These are new columns being added.
    DIET_plant_supply[y] = DIET_plant_supply['GrandTotal'] * fact / (1-DIET_plant_supply['Post-farm gate loss'])
    
# DIET_demand[@2014]*VLOOKUP(@DIET_waste[@[Waste lookup]:[Waste lookup]],waste_forecast[#All],11,0)/(1-VLOOKUP(@DIET_waste[@[Waste lookup]:[Waste lookup]],waste_forecast[#All],11,0))
# DIET_waste = DIET_demand.merge(waste_forecast['Post-farm gate loss'])
DIET_waste = DIET_demand.join(countries['Waste region'])
idx = DIET_waste.index
DIET_waste = pd.merge(DIET_waste, waste_forecast['Post-farm gate loss'], on=['Waste region', 'Waste category'], how='left')
DIET_waste.set_index(idx,inplace=True)

for y in years:
    DIET_waste[y] = DIET_waste[y] * DIET_waste['Post-farm gate loss']/(1-DIET_waste['Post-farm gate loss'])

DIET_supply = DIET_demand[years] + DIET_waste[years]
# DIET_waste, DIET_demand and DIET_supply are now correct.


In [4]:
# Per-capita PDS total food supply forecast, 2014-2100 [kcal/(cap-day)]
# PDS_supply from "1 - Food System CORE 75 6-18-21.xlsx".

# PDS_demand:

arr = []
start_yr = parameters['diet_start_year']
end_yr = parameters['diet_end_year']
adoption = parameters['plant_rich_diet_adoption']
denominator = end_yr - start_yr + 1
for y in range(start_yr, end_yr+1):
    val =  adoption * (y - start_yr +1) / denominator
    arr.append(val)

for y in range(end_yr, years_int[-1]):
    arr.append(adoption)

for y in range(years_int[0],start_yr):
    arr.insert(0,0.0)

PDS_demand = REF_demand.copy()
for y in years:
    fact = arr.pop(0)
    PDS_demand[y] = PDS_demand[y] * (1-fact) + plantrich_demand['GrandTotal'] * fact

# PDS_demand now complete

# next for PDS_waste

PDS_waste = PDS_demand.join(food_lookup['Waste category']).join(countries['Waste region'])
w_suffix, f_suffix = ('_waste','_forecast')
idx = PDS_waste.index
PDS_waste = pd.merge(PDS_waste, waste_forecast.reset_index(), on=['Waste category', 'Waste region'], how='left', suffixes=[w_suffix, f_suffix])
PDS_waste.set_index(idx, inplace=True)

for y in years:
    w_idx = y + w_suffix
    f_idx = y + f_suffix
    PDS_waste[y] = PDS_waste[w_idx] * PDS_waste[f_idx] / (1-PDS_waste[f_idx])
    
PDS_waste = PDS_waste[years]

# PDS_waste now complete
PDS_supply = PDS_waste + PDS_demand


In [5]:
# PDS food supply forecast *plant-rich diet only*, 2014-2100 [kcal/(cap-day)]
# PDS_plant_supply table on the PDS_supply tab in the "1 - Food System CORE 75 6-18-21.xlsx" workbook.
# formula e.g. PDS_plant_supply = plantrich_demand[@2017]*PDS_demand!G$2/(1-INDEX(waste_forecast[#All],MATCH(@PDS_plant_supply[@[Waste lookup]:[Waste lookup]],waste_forecast[[#All],[Waste lookup]:[Waste lookup]],0),MATCH(PDS_plant_supply[[#Headers],[2017]],waste_forecast[#Headers],0)))

# pds_supply_plant_rich = pd.read_csv(r'/mnt/c/Users/neilm/Documents/Drawdown/Excel Files/food_system/data/PDS_food_supply_plant_rich_diet.txt', index_col=['Country', 'FAO commodity'])

# PDS_plant_supply = pds_supply_plant_rich comes from plantrich_demand, which comes from plant_rich_diet, which is read in from a file, which represents the controversial table with GrandTotal.

#PDS_plant_supply
PDS_plant_supply = PDS_demand.join(food_lookup['Waste category']).join(countries['Waste region'])

d_suffix, f_suffix = ('_demand','_forecast')
idx = PDS_plant_supply.index
PDS_plant_supply = pd.merge(PDS_plant_supply, waste_forecast.reset_index(), on=['Waste category', 'Waste region'], how='left', suffixes=[d_suffix, f_suffix])
PDS_plant_supply.set_index(idx, inplace=True)


arr = []
start_yr = parameters['diet_start_year']
end_yr = parameters['diet_end_year']
adoption = parameters['plant_rich_diet_adoption']
denominator = end_yr - start_yr + 1
for y in range(start_yr, end_yr+1):
    val =  adoption * (y - start_yr +1) / denominator
    arr.append(val)

for y in range(end_yr, years_int[-1]):
    arr.append(adoption)

for y in range(years_int[0],start_yr):
    arr.insert(0,0.0)

for y in years:
    fact = arr.pop(0)
    f_idx = y + f_suffix
    PDS_plant_supply[y] = plantrich_demand['GrandTotal'] * fact / (1-PDS_plant_supply[f_idx])

PDS_plant_supply = PDS_plant_supply[years]
# PDS_plant_supply now matches PDS_plant_supply table on the PDS_supply tab in the "1 - Food System CORE 75 6-18-21.xlsx" workbook.

In [6]:
# Per-capita WASTE-only total food supply forecast, 2014-2100 [kcal/(cap-day)]
# WASTE_supply from "1 - Food System CORE 75 6-18-21.xlsx".

# formula e.g. =SUM(WASTE_waste[@2017],WASTE_demand[@2017])
# waste_supply = pd.read_csv(r'/mnt/c/Users/neilm/Documents/Drawdown/Excel Files/food_system/data/WASTE_supply.txt', index_col=['Country', 'FAO commodity'])

# Need to calculate WASTE_waste and WASTE_demand # Note: WASTE_demand = REF_demand in the spreadsheet
# To calculate WASTE_waste:
# WASTE_demand[@2014]*INDEX(waste_forecast[#All],MATCH(@WASTE_waste[@[Waste lookup]:[Waste lookup]],waste_forecast[[#All],[Waste lookup]:[Waste lookup]],0),MATCH(WASTE_waste[[#Headers],[2014]],waste_forecast[#Headers],0))/(1-INDEX(waste_forecast[#All],MATCH(@WASTE_waste[@[Waste lookup]:[Waste lookup]],waste_forecast[[#All],[Waste lookup]:[Waste lookup]],0),MATCH(WASTE_waste[[#Headers],[2014]],waste_forecast[#Headers],0)))


WASTE_waste = REF_demand.join(food_lookup['Waste category']).join(countries['Waste region'])
w_suffix, f_suffix = ('_waste','_forecast')
idx = WASTE_waste.index
WASTE_waste = pd.merge(WASTE_waste, waste_forecast.reset_index(), on=['Waste category', 'Waste region'], how='left', suffixes=[w_suffix, f_suffix])
WASTE_waste.set_index(idx, inplace=True)

old_cols = []
for y in years:
    w_idx = y + w_suffix
    f_idx = y + f_suffix
    WASTE_waste[y] = WASTE_waste[w_idx] * WASTE_waste[f_idx] / (1-WASTE_waste[f_idx])
    old_cols.append(w_idx)
    old_cols.append(f_idx)
    
#WASTE_waste.drop(labels=old_cols, axis='columns', inplace=True)
#WASTE_waste = WASTE_waste.set_index(['Waste category', 'Waste region'])
WASTE_waste = WASTE_waste[years]

WASTE_supply = WASTE_waste + REF_demand
#food_waste_lookup

In [7]:
# REF_emissions
REF_emissions_work = REF_supply.copy()
REF_emissions_work.drop(labels=drop_countries, level=0, axis='rows', inplace=True)

REF_emissions_work = REF_emissions_work.loc[:,years].multiply(population_high)
REF1_emissions = LCA_emissions_by_product.join(other=REF_emissions_work)
avg = REF1_emissions['Avg']

for y in years:
    REF1_emissions[y] *= avg * 365 / 10**6

REF2_emissions_work = REF_supply.copy()
REF2_emissions_work.drop(labels=drop_countries, level=0, axis='rows', inplace=True)

REF2_emissions_work = REF2_emissions_work.loc[:,years].multiply(population_medium)
REF2_emissions = LCA_emissions_by_product.join(other=REF2_emissions_work)
avg = REF2_emissions['Avg']

for y in years:
    REF2_emissions[y] *= avg * 365 / 10**6


In [8]:
# PDS_emissions

diet_diff = DIET_supply.copy() # old = diet_supply_core
diet_supply_plant_rich_work = DIET_plant_supply.copy() # old = diet_supply_plant_rich

diet_diff.drop(labels=drop_countries, level=0, axis='rows', inplace=True)
#diet_supply_plant_rich_work.drop(labels=drop_countries, level=0, axis='rows', inplace=True)

for y in years:
    diet_diff[y] -= diet_supply_plant_rich_work[y]
 
avg = LCA_emissions_by_product['Avg']
diet_supply_core_work = diet_diff.copy()
diet_supply_core_work2 = DIET_plant_supply.copy() # old = diet_supply_plant_rich
#diet_supply_core_work2.drop(labels=drop_countries, level=0, axis='rows', inplace=True)

for y in years:
    diet_supply_core_work[y] *= avg
    diet_supply_core_work2[y] *= avg
    
    diet_supply_core_work[y] *= population_medium[y]
    diet_supply_core_work2[y] *= population_medium[y]
    if int(y) >= parameters['diet_start_year']:
        diet_supply_core_work2[y] *= (1-parameters['localization_emisssion_reduction_percentage'])

diet_annual_emissions = (diet_supply_core_work + diet_supply_core_work2) * 365/10**6

# calculate waste emissions:
waste_emissions = WASTE_supply.copy()
waste_emissions.drop(labels=drop_countries, level=0, axis='rows', inplace=True)
for y in years:
    waste_emissions[y] *= avg
    waste_emissions[y] *= population_medium[y]
    waste_emissions[y] *= 365/(10**6)

# calculate PDS annual emissions:
pds_diff = PDS_supply.copy() # old: pds_supply_core
pds_diff.drop(labels=drop_countries, level=0, axis='rows', inplace=True)
for y in years:
    pds_diff[y] -= PDS_plant_supply[y] # old = pds_supply_plant_rich[y]
    
pds_supply_core_work = pds_diff.copy()
pds_supply_core_work.drop(labels=drop_countries, level=0, axis='rows', inplace=True)
pds_supply_core_work2 = PDS_plant_supply.copy() # old = pds_supply_plant_rich
pds_supply_core_work2.drop(labels=drop_countries, level=0, axis='rows', inplace=True)

for y in years:
    pds_supply_core_work[y] *= avg
    pds_supply_core_work2[y] *= avg
    
    pds_supply_core_work[y] *= population_medium[y]
    pds_supply_core_work2[y] *= population_medium[y]
    if int(y) >= parameters['diet_start_year']:
        pds_supply_core_work2[y] *= (1-parameters['localization_emisssion_reduction_percentage'])

pds_annual_emissions = (pds_supply_core_work + pds_supply_core_work2) * 365/10**6


In [9]:
# REF_country_CO2
REF1_country_per_capita_emissions = REF1_emissions.loc[:,years].groupby(level='Country').sum()
for y in years:
    REF1_country_per_capita_emissions[y] /= (population_high[y] * 365 / (10**3) )

REF2_country_per_capita_emissions = REF2_emissions.loc[:,years].groupby(level='Country').sum()
for y in years:
     REF2_country_per_capita_emissions[y] /= (population_medium[y] * 365 / (10**3) )


In [10]:
# PDS_country_CO2
pds_diet_per_capita_emissions = diet_annual_emissions.groupby(level='Country').sum()
waste_per_capita_emissions = waste_emissions.groupby(level='Country').sum()
pds_per_capita_emissions = pds_annual_emissions.groupby(level='Country').sum()

for y in years:
    population = (population_medium[y] * 365 / (10**3) )
    pds_diet_per_capita_emissions[y] /= population
    waste_per_capita_emissions[y] /= population
    pds_per_capita_emissions[y] /= population

# Note: [NL, 14 Oct 2021], the following three tables have incorrect formulas in the spreadsheet:
# pds_diet_per_capita_emissions, waste_per_capita_emissions, pds_per_capita_emissions


In [11]:
# REF_commodity_CO2
REF1_commodity_CO2 = REF1_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.
REF2_commodity_CO2 = REF2_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.


In [12]:
# PDS_commodity_CO2
diet_commodity_CO2 = diet_annual_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.
waste_commodity_CO2 = waste_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.
pds_commodity_CO2 = pds_annual_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.


In [13]:
# Global yield

REF_supply_year_yield_pop_ref1 = REF_supply.copy()
REF_supply_year_yield_pop_ref2 = REF_supply.copy()
diet_supply_year_yield_pop_ref2 = DIET_supply.join(food_waste_lookup['Yield category']) # old: diet_supply_core
waste_supply_year_yield_pop_ref2 = WASTE_supply.join(food_waste_lookup['Yield category']) # old: waste_supply
pds_supply_year_yield_pop_ref2 = PDS_supply.join(food_waste_lookup['Yield category']) # old: pds_supply_core

kcal_per_gram = REF_supply_year_yield_pop_ref1['kcal/g']
for y in years:
    REF_supply_year_yield_pop_ref1[y] /= kcal_per_gram
    REF_supply_year_yield_pop_ref1[y] *= population_high[y]
    REF_supply_year_yield_pop_ref1[y] *= 365
    
    REF_supply_year_yield_pop_ref2[y] /= kcal_per_gram
    REF_supply_year_yield_pop_ref2[y] *= population_medium[y]
    REF_supply_year_yield_pop_ref2[y] *= 365
    
    diet_supply_year_yield_pop_ref2[y] /= kcal_per_gram
    diet_supply_year_yield_pop_ref2[y] *= population_medium[y]
    diet_supply_year_yield_pop_ref2[y] *= 365
    
    waste_supply_year_yield_pop_ref2[y] /= kcal_per_gram
    waste_supply_year_yield_pop_ref2[y] *= population_medium[y]
    waste_supply_year_yield_pop_ref2[y] *= 365
    
    pds_supply_year_yield_pop_ref2[y] /= kcal_per_gram
    pds_supply_year_yield_pop_ref2[y] *= population_medium[y]
    pds_supply_year_yield_pop_ref2[y] *= 365

report_dict = {'Grain': {'Cereals', 'Fruits and vegetables', 'Oilseeds and pulses', 'Roots and tubers', 'Other'},
               'Meat': {'Bovine Meat', 'Meat, Other', 'Mutton & Goat Meat', 'Pigmeat', 'Poultry Meat', 'Fats, Animals, Raw', 'Offals, Edible'},
               'Milk and egg': {'Butter, Ghee', 'Cream', 'Eggs', 'Milk - Excluding Butter', 'Whey'},
               'Fish and seafood': {'Aquatic Animals, Others', 'Aquatic Plants', 'Cephalopods', 'Crustaceans', 'Demersal Fish', 'Fish, Body Oil', 'Fish, Liver Oil',
                                    'Freshwater Fish', 'Marine Fish, Other',  'Meat, Aquatic Mammals', 'Molluscs, Other', 'Pelagic Fish'}
               }

REF_supply_year_yield_pop_ref1_yc = REF_supply_year_yield_pop_ref1.groupby('Yield category').sum()
REF_supply_year_yield_pop_ref1_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

REF_supply_year_yield_pop_ref2_yc = REF_supply_year_yield_pop_ref2.groupby('Yield category').sum()
REF_supply_year_yield_pop_ref2_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

diet_supply_year_yield_pop_ref2_yc = diet_supply_year_yield_pop_ref2.groupby('Yield category').sum()
diet_supply_year_yield_pop_ref2_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

waste_supply_year_yield_pop_ref2_yc = waste_supply_year_yield_pop_ref2.groupby('Yield category').sum()
waste_supply_year_yield_pop_ref2_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

pds_supply_year_yield_pop_ref2_yc = pds_supply_year_yield_pop_ref2.groupby('Yield category').sum()
pds_supply_year_yield_pop_ref2_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

idx_vals = set(REF_supply_year_yield_pop_ref1_yc.index.to_list())

for report_category, yield_categories in report_dict.items():
    REF_supply_year_yield_pop_ref1_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    REF_supply_year_yield_pop_ref2_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    diet_supply_year_yield_pop_ref2_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    waste_supply_year_yield_pop_ref2_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    pds_supply_year_yield_pop_ref2_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category


In [14]:
# Supply_ADOPTION_DATE

# why are each of these are identical?
supply_results = {}

ref1_series = REF_supply_year_yield_pop_ref1_yc.groupby('Report Category').sum() / 10**6
ref1_series.loc['Meat'] += ref1_series.loc['Milk and egg']
ref1_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)

ref2_series = REF_supply_year_yield_pop_ref2_yc.groupby('Report Category').sum() / 10**6
ref2_series.loc['Meat'] += ref2_series.loc['Milk and egg']
ref2_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)
 
diet_series = diet_supply_year_yield_pop_ref2_yc.groupby('Report Category').sum() / 10**6
diet_series.loc['Meat'] += diet_series.loc['Milk and egg']
diet_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)

waste_series = waste_supply_year_yield_pop_ref2_yc.groupby('Report Category').sum() / 10**6
waste_series.loc['Meat'] += waste_series.loc['Milk and egg']
waste_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)

pds_series = pds_supply_year_yield_pop_ref2_yc.groupby('Report Category').sum() / 10**6
pds_series.loc['Meat'] += pds_series.loc['Milk and egg']
pds_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)


ref1_ref2_diff = ref1_series.loc[:,years] - ref2_series.loc[:,years]
a = ref1_ref2_diff.loc['Grain', '2015':'2060'].sum() / 10**9
b = ref1_ref2_diff.loc['Meat', '2015':'2060'].sum() / 10**9
c = ref1_ref2_diff.loc['Fish and seafood', '2015':'2060'].sum() / 10**9
supply_results['Family Planning'] = (a,b,c)

ref2_diet_diff = ref2_series.loc[:,years] - diet_series.loc[:,years]
d = ref2_diet_diff.loc['Grain', '2015':'2060'].sum() / 10**9
e = ref2_diet_diff.loc['Meat', '2015':'2060'].sum() / 10**9
f = ref2_diet_diff.loc['Fish and seafood', '2015':'2060'].sum() / 10**9
supply_results['Plant-rich Diet only (75%)'] = (d,e,f)

ref2_waste_diff = ref2_series.loc[:,years] - waste_series.loc[:,years]
g = ref2_waste_diff.loc['Grain', '2015':'2060'].sum() / 10**9
h = ref2_waste_diff.loc['Meat', '2015':'2060'].sum() / 10**9
i = ref2_waste_diff.loc['Fish and seafood', '2015':'2060'].sum() / 10**9
supply_results['Food waste only (75%)'] = (g,h,i)

ref2_pds_diff = ref2_series.loc[:,years] - pds_series.loc[:,years]
j = ref2_pds_diff.loc['Grain', '2015':'2060'].sum() / 10**9
k = ref2_pds_diff.loc['Meat', '2015':'2060'].sum() / 10**9
m = ref2_pds_diff.loc['Fish and seafood', '2015':'2060'].sum() / 10**9
supply_results['Integrated Food Solutions (Diet + Waste)'] = (j,k,m)

supply_results['Percent of yield reductions from diet soln'] = (100*d/j, 100*e/k, 100*f/m)
supply_results['Percent of yield reductions from waste soln'] = (100*(1- d/j),100*(1- e/k),100*(1- f/m))

report = pd.DataFrame(supply_results).transpose()
report.columns = ('crop-based reduction','livestock reduction','seafood reduction')
report



,crop-based reduction,livestock reduction,seafood reduction
Family Planning,16.249240,4.341611,0.587320
Plant-rich Diet only (75%),33.676666,11.784905,1.733539
Food waste only (75%),49.828375,6.131898,1.483911
Integrated Food Solutions (Diet + Waste),78.234277,16.561784,2.920955
Percent of yield reductions from diet soln,43.045922,71.157223,59.348345
Percent of yield reductions from waste soln,56.954078,28.842777,40.651655


In [15]:
# Yield_ADOPTION_DATE
yield_results = {}
ref1_series = REF_supply_year_yield_pop_ref1.sum()
ref2_series = REF_supply_year_yield_pop_ref2.sum()
diet_series = diet_supply_year_yield_pop_ref2.sum()
waste_series = waste_supply_year_yield_pop_ref2.sum()
pds_series = pds_supply_year_yield_pop_ref2.sum()

ref1_ref2_diff = ref1_series.loc[years] - ref2_series.loc[years]
a = ref1_ref2_diff.loc['2020':'2050'].sum() / 10**15
b = ref1_ref2_diff.loc['2015':'2060'].sum() / 10**15
yield_results['Family Planning'] = (a,b)

ref2_diet_diff = ref2_series.loc[years] - diet_series.loc[years]
c = ref2_diet_diff.loc['2020':'2050'].sum() / 10**15
d = ref2_diet_diff.loc['2015':'2060'].sum() / 10**15
yield_results['Plant-rich Diet only (75%)'] = (c,d)

ref2_waste_diff = ref2_series.loc[years] - waste_series.loc[years]
e = ref2_waste_diff.loc['2020':'2050'].sum() / 10**15
f = ref2_waste_diff.loc['2015':'2060'].sum() / 10**15
yield_results['Food waste only (75%)'] = (e,f)

ref2_pds_diff = ref2_series.loc[years] - pds_series.loc[years]
g = ref2_pds_diff.loc['2020':'2050'].sum() / 10**15
h = ref2_pds_diff.loc['2015':'2060'].sum() / 10**15
yield_results['Integrated Food Solutions (Diet + Waste)'] = (g,h)

yield_results['Percent of yield reductions from diet soln'] = (100*c/g, 100*d/h)
yield_results['Percent of yield reductions from waste soln'] = (100*(1- c/g),100*(1- d/h))

report = pd.DataFrame(yield_results).transpose()
report.columns = ('2020-50 cumulative reduction','2015-60 cumulative reduction')
report

# Emissions_ADOPTION_DATE


,2020-50 cumulative reduction,2015-60 cumulative reduction
Family Planning,10.427293,21.178171
Plant-rich Diet only (75%),28.357596,47.195110
Food waste only (75%),35.564429,57.444184
Integrated Food Solutions (Diet + Waste),60.200862,97.717016
Percent of yield reductions from diet soln,47.104967,48.297740
Percent of yield reductions from waste soln,52.895033,51.702260


In [16]:
# Emissions_ADOPTION_DATE

emissions_results = {}
ref1_series = REF1_emissions.sum()
ref2_series = REF2_emissions.sum()
diet_series = diet_annual_emissions.sum()
waste_series = waste_emissions.sum()
pds_series = pds_annual_emissions.sum()

ref1_ref2_diff = ref1_series.loc[years] - ref2_series.loc[years]
a = ref1_ref2_diff.loc['2020':'2050'].sum() / 10**9
b = ref1_ref2_diff.loc['2015':'2060'].sum() / 10**9
emissions_results['Family Planning'] = (a,b)

ref2_diet_diff = ref2_series.loc[years] - diet_series.loc[years]
c = ref2_diet_diff.loc['2020':'2050'].sum() / 10**9
d = ref2_diet_diff.loc['2015':'2060'].sum() / 10**9
emissions_results['Plant-rich Diet only (75%)'] = (c,d)

ref2_waste_diff = ref2_series.loc[years] - waste_series.loc[years]
e = ref2_waste_diff.loc['2020':'2050'].sum() / 10**9
f = ref2_waste_diff.loc['2015':'2060'].sum() / 10**9
emissions_results['Food waste only (75%)'] = (e,f)

ref2_pds_diff = ref2_series.loc[years] - pds_series.loc[years]
g = ref2_pds_diff.loc['2020':'2050'].sum() / 10**9
h = ref2_pds_diff.loc['2015':'2060'].sum() / 10**9
emissions_results['Integrated Food Solutions (Diet + Waste)'] = (g,h)

emissions_results['Percent of yield reductions from diet soln'] = (100*c/g, 100*d/h)
emissions_results['Percent of yield reductions from waste soln'] = (100*(1- c/g),100*(1- d/h))

report = pd.DataFrame(emissions_results).transpose()
report.columns = ('2020-50 cumulative reduction','2015-60 cumulative reduction')
report

,2020-50 cumulative reduction,2015-60 cumulative reduction
Family Planning,21.595881,44.327116
Plant-rich Diet only (75%),108.851472,183.747467
Food waste only (75%),48.026016,78.483813
Integrated Food Solutions (Diet + Waste),144.667316,239.362795
Percent of yield reductions from diet soln,75.242616,76.765258
Percent of yield reductions from waste soln,24.757384,23.234742


In [17]:
#The datasets calculated below match the tables on their respective worksheet tabs in the s/sht.
# Note that the s/sht tables are not updated properly. S/sht formulas need to be copied down and recalculated.

#REF_country_supply
REF_supply_year_yield_pop_ref1.groupby('Country').sum()/10**6
REF_supply_year_yield_pop_ref2.groupby('Country').sum()/10**6

#PDS_country_supply
diet_supply_year_yield_pop_ref2.groupby('Country').sum()/10**6
waste_supply_year_yield_pop_ref2.groupby('Country').sum()/10**6
pds_supply_year_yield_pop_ref2.groupby('Country').sum()/10**6

# REF_commodity_supply
REF_supply_year_yield_pop_ref1.groupby('FAO commodity').sum()/10**6
REF_supply_year_yield_pop_ref2.groupby('FAO commodity').sum()/10**6

diet_supply_year_yield_pop_ref2.groupby('FAO commodity').sum()/10**6
waste_supply_year_yield_pop_ref2.groupby('FAO commodity').sum()/10**6
pds_supply_year_yield_pop_ref2.groupby('FAO commodity').sum()/10**6


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
FAO commodity,,,,,,,,,,,,,,,,,,,,,
Apples and products,8.881508e+07,8.765846e+07,9.214107e+07,9.546497e+07,9.881736e+07,9.855238e+07,9.832847e+07,9.793814e+07,9.749369e+07,9.700000e+07,...,5.913880e+07,5.879069e+07,5.844200e+07,5.809249e+07,5.774191e+07,5.738997e+07,5.703634e+07,5.668066e+07,5.632254e+07,5.596156e+07
"Aquatic Animals, Others",0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Aquatic Plants,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Bananas,1.295215e+08,1.373460e+08,1.315526e+08,1.333276e+08,1.341226e+08,1.354104e+08,1.367539e+08,1.379739e+08,1.391471e+08,1.402731e+08,...,1.682499e+08,1.684754e+08,1.686901e+08,1.688943e+08,1.690885e+08,1.692727e+08,1.694473e+08,1.696123e+08,1.697677e+08,1.699134e+08
Barley and products,8.809886e+06,9.004841e+06,8.313149e+06,8.484374e+06,9.302787e+06,9.365131e+06,9.425108e+06,9.479216e+06,9.530197e+06,9.578294e+06,...,1.110709e+07,1.110593e+07,1.110350e+07,1.109981e+07,1.109485e+07,1.108863e+07,1.108112e+07,1.107232e+07,1.106223e+07,1.105082e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tomatoes and products,2.019491e+08,2.055738e+08,2.037563e+08,2.097998e+08,2.130265e+08,2.130965e+08,2.131412e+08,2.128909e+08,2.125507e+08,2.121289e+08,...,1.808150e+08,1.807740e+08,1.807225e+08,1.806601e+08,1.805866e+08,1.805016e+08,1.804046e+08,1.802950e+08,1.801722e+08,1.800356e+08
"Vegetables, Other",1.007727e+09,1.034172e+09,1.055736e+09,1.069255e+09,1.088399e+09,1.087761e+09,1.088766e+09,1.088255e+09,1.087062e+09,1.085238e+09,...,7.852168e+08,7.825157e+08,7.797997e+08,7.770660e+08,7.743117e+08,7.715331e+08,7.687263e+08,7.658870e+08,7.630106e+08,7.600919e+08
Wheat and products,5.641199e+08,5.714513e+08,5.831625e+08,5.925542e+08,5.971897e+08,5.946155e+08,5.923901e+08,5.895502e+08,5.865389e+08,5.833757e+08,...,4.117908e+08,4.103173e+08,4.088241e+08,4.073115e+08,4.057792e+08,4.042268e+08,4.026537e+08,4.010591e+08,3.994419e+08,3.978005e+08


In [18]:
### Start waste calculations. ###

# Percent of food supply remaining after each step in food life cycle.

# Note on 'Agricultural Production' column:
# Because agricultural production losses occur before the farm gate -- "supply" in the Food System model is measured at the farm gate, 
# and considered to represent '100%' -- values here are conceptualized in this model as the volume of supply (in 'excess' of 100%) that *would have been* present
# if there had been no agricultural production losses.
waste_remains = waste_forecast.copy()
waste_remains['Agricultural Production'] = 1/(1-waste_remains['Agricultural Production'])
waste_remains['Postharvest handling and storage'] = 1-waste_remains['Postharvest handling and storage']
waste_remains['Processing'] = waste_remains['Postharvest handling and storage'] * (1-waste_remains['Processing'])
waste_remains['Packaging'] = waste_remains['Processing']*(1-waste_remains['Packaging'])
waste_remains['Distribution'] = waste_remains['Packaging']*(1-waste_remains['Distribution'])
waste_remains['Consumption'] = waste_remains['Distribution'] *(1-waste_remains['Consumption'])
waste_remains['Post-farm gate loss'] = (1-waste_remains['Consumption'])

#Mass percent of post-farm gate food loss/waste stream by region and waste category
waste_percent = waste_remains.copy()
waste_percent['Agricultural Production'] = (waste_remains['Agricultural Production']-1)/waste_remains['Post-farm gate loss']
waste_percent['Postharvest handling and storage'] = (1-waste_remains['Postharvest handling and storage'])/waste_remains['Post-farm gate loss']
waste_percent['Processing'] = (waste_remains['Postharvest handling and storage'] - waste_remains['Processing'])/waste_remains['Post-farm gate loss']
waste_percent['Packaging'] = (waste_remains['Processing'] - waste_remains['Packaging'])/waste_remains['Post-farm gate loss'] 
waste_percent['Distribution'] = (waste_remains['Packaging'] - waste_remains['Distribution'])/waste_remains['Post-farm gate loss']
waste_percent['Consumption'] = (waste_remains['Distribution'] -waste_remains['Consumption'])/waste_remains['Post-farm gate loss']
waste_percent.drop(labels=['Post-farm gate loss'], axis='columns', inplace=True)
waste_percent['Check'] = waste_percent.sum(axis='columns') - waste_percent['Agricultural Production']


In [19]:
REF_waste_mass = REF_waste.join(countries['Waste region'])
food_waste_kcal_g = food_waste_lookup['kcal/g']
food_kcal_g = food_lookup['kcal/g']
DIET_waste_mass = DIET_waste.copy()
WASTE_waste_mass = WASTE_waste.join(countries['Waste region']).join(food_lookup['Waste category'])
PDS_waste_mass = PDS_waste.join(countries['Waste region']).join(food_lookup['Waste category'])

for y in years:
    REF_waste_mass[y] = REF_waste_mass[y] / food_waste_kcal_g
    DIET_waste_mass[y] = DIET_waste_mass[y] / food_kcal_g
    WASTE_waste_mass[y] = WASTE_waste_mass[y] / food_kcal_g
    PDS_waste_mass[y] = PDS_waste_mass[y] / food_kcal_g
    
REF_waste_mass.reset_index(inplace=True)
REF_waste_mass.drop(labels='FAO commodity', axis='columns', inplace=True)
new_idx = ['Waste region','Country','Waste category']

REF_waste_mass = REF_waste_mass.groupby(new_idx).sum()

DIET_waste_mass.reset_index(inplace=True)
DIET_waste_mass.drop(labels='FAO commodity', axis='columns', inplace=True)
DIET_waste_mass = DIET_waste_mass.groupby(new_idx).sum()
DIET_waste_mass = DIET_waste_mass[years]

WASTE_waste_mass.reset_index(inplace=True)
WASTE_waste_mass.drop(labels='FAO commodity', axis='columns', inplace=True)
WASTE_waste_mass = WASTE_waste_mass.groupby(new_idx).sum()
WASTE_waste_mass = WASTE_waste_mass[years]

PDS_waste_mass.reset_index(inplace=True)
PDS_waste_mass.drop(labels='FAO commodity', axis='columns', inplace=True)
PDS_waste_mass = PDS_waste_mass.groupby(new_idx).sum()
PDS_waste_mass = PDS_waste_mass[years]

# # Per-capita REF food waste forecast, 2014-2100 [g/(cap-day)]

# Country life cycle waste fractions (in tonnes, by country, waste category, life cycle stage, and year) are
# the sum of all waste (in mass per capita per day) for a given country in a given waste category multiplied by the life cycle waste fraction
# for the given life cycle stage, multiplied by the population of a given country, by 365 to generate an annual value, and by 10^6 to convert from g to tonnes


In [20]:
import time

pf_one = time.perf_counter()

waste_categories = ['Agricultural Production','Postharvest handling and storage','Processing','Packaging','Distribution','Consumption']
column_index = pd.MultiIndex.from_product([years,waste_categories])

REF1_fraction = REF_waste_mass.copy()
REF2_fraction = REF_waste_mass.copy()
DIET_fraction = DIET_waste_mass.copy()
WASTE_fraction = WASTE_waste_mass.copy()
PDS_fraction = PDS_waste_mass.copy()

population_high_with_region = countries.join(other=population_high).reset_index()
population_high_with_region.set_index(['Waste region','Country'], inplace=True)

population_medium_with_region = countries.join(other=population_medium).reset_index()
population_medium_with_region.set_index(['Waste region','Country'], inplace=True)

for y in years:
    REF1_fraction.loc[:,y] *= population_high_with_region[y] * 365 / 10**6
    REF2_fraction.loc[:,y] *= population_medium_with_region[y] * 365 / 10**6
    DIET_fraction.loc[:,y] *= population_medium_with_region[y] * 365 / 10**6
    WASTE_fraction.loc[:,y] *= population_medium_with_region[y] * 365 / 10**6
    PDS_fraction.loc[:,y] *= population_medium_with_region[y] * 365 / 10**6


REF1_fraction = REF1_fraction[years].reindex(columns=column_index, level=0) # broadcast across (year, waste category) entries
REF2_fraction = REF2_fraction[years].reindex(columns=column_index, level=0)
DIET_fraction = DIET_fraction[years].reindex(columns=column_index, level=0)
WASTE_fraction = WASTE_fraction[years].reindex(columns=column_index, level=0)
PDS_fraction = PDS_fraction[years].reindex(columns=column_index, level=0)

dfs = [REF1_fraction, REF2_fraction, DIET_fraction, WASTE_fraction, PDS_fraction]

waste_percent_wk = waste_percent[waste_categories].swaplevel()
pf_two = time.perf_counter()

for i,df in enumerate(dfs):
    idx = df.index.names
    df.reset_index(inplace=True)
    df.set_index(['Waste region', 'Waste category', 'Country'], inplace=True)
    
    for y in years:
        for wc in waste_categories:
            df.loc[:,(y,wc)] *= waste_percent_wk[wc]

    df.reset_index(inplace=True)
    df.set_index(idx, inplace=True)

pf_three = time.perf_counter()

REF1_fraction_category = REF1_fraction.groupby(level=['Waste category']).sum()
REF2_fraction_category = REF2_fraction.groupby(level=['Waste category']).sum()
DIET_fraction_category = DIET_fraction.groupby(level=['Waste category']).sum()
WASTE_fraction_category = WASTE_fraction.groupby(level=['Waste category']).sum()
PDS_fraction_category = PDS_fraction.groupby(level=['Waste category']).sum()

pf_four = time.perf_counter()

print(pf_one,pf_two,pf_three,pf_four)


13508.2563763 13510.6175209 13522.6402991 13522.6520192


In [21]:
REF1_fraction_category_report = REF1_fraction_category.groupby(level=[0],axis='columns').sum()
REF2_fraction_category_report = REF2_fraction_category.groupby(level=[0],axis='columns').sum()
DIET_fraction_category_report = DIET_fraction_category.groupby(level=[0],axis='columns').sum()
WASTE_fraction_category_report = WASTE_fraction_category.groupby(level=[0],axis='columns').sum()
PDS_fraction_category_report = PDS_fraction_category.groupby(level=[0],axis='columns').sum()

REF1_fraction_yearly_report = REF1_fraction_category.sum().groupby(level=0).sum()
REF2_fraction_yearly_report = REF2_fraction_category.sum().groupby(level=0).sum()
DIET_fraction_yearly_report = DIET_fraction_category.sum().groupby(level=0).sum()
WASTE_fraction_yearly_report = WASTE_fraction_category.sum().groupby(level=0).sum()
PDS_fraction_yearly_report = PDS_fraction_category.sum().groupby(level=0).sum()

family_planning_report = REF1_fraction_yearly_report - REF2_fraction_yearly_report
diet_report = REF2_fraction_yearly_report - DIET_fraction_yearly_report
waste_report = REF2_fraction_yearly_report - WASTE_fraction_yearly_report
pds_report = REF2_fraction_yearly_report - PDS_fraction_yearly_report


In [22]:

ref1_input1 = REF1_fraction_category_report.loc[:,'2020':'2050'].sum().sum() / 10**9
ref2_input1 = REF2_fraction_category_report.loc[:,'2020':'2050'].sum().sum() / 10**9
diet_input1 = DIET_fraction_category_report.loc[:,'2020':'2050'].sum().sum() / 10**9
waste_input1 = WASTE_fraction_category_report.loc[:,'2020':'2050'].sum().sum() / 10**9
pds_input1 = PDS_fraction_category_report.loc[:,'2020':'2050'].sum().sum() / 10**9

ref1_input2 = REF1_fraction_category_report.loc[:,'2015':'2060'].sum().sum() / 10**9
ref2_input2 = REF2_fraction_category_report.loc[:,'2015':'2060'].sum().sum() / 10**9
diet_input2 = DIET_fraction_category_report.loc[:,'2015':'2060'].sum().sum() / 10**9
waste_input2 = WASTE_fraction_category_report.loc[:,'2015':'2060'].sum().sum() / 10**9
pds_input2 = PDS_fraction_category_report.loc[:,'2015':'2060'].sum().sum() / 10**9

ref1_input3 = family_planning_report.loc['2020':'2050'].sum() / 10**9
ref2_input3 = diet_report.loc['2020':'2050'].sum() / 10**9
diet_input3 = waste_report.loc['2020':'2050'].sum() / 10**9
waste_input3 = pds_report.loc['2020':'2050'].sum() / 10**9

ref1_input4 = family_planning_report.loc['2015':'2060'].sum() / 10**9
ref2_input4 = diet_report.loc['2015':'2060'].sum() / 10**9
diet_input4 = waste_report.loc['2015':'2060'].sum() / 10**9
waste_input4 = pds_report.loc['2015':'2060'].sum() / 10**9

report = {'Solutions implemented': ['Family Planning', 'Plant-rich Diet ONLY (75%)', 'Food Waste ONLY (75%)', 'Integrated Food Solutions (Diet + Waste)'],
        'Scenario comparison': ['REF1-REF2', 'REF2-DIET', 'REF2-WASTE', 'REF2-PDS'],
        '2020-50 cumulative reduction': [ref1_input1-ref2_input1, ref2_input1-diet_input1, ref2_input1-waste_input1, ref2_input1-pds_input1],
        '2015-60 cumulative reduction': [ref1_input2-ref2_input2, ref2_input2-diet_input2, ref2_input2-waste_input2, ref2_input2-pds_input2],
        '2020-50 cumulative MSW reduction': [ref1_input3, ref2_input3, diet_input3, waste_input3],
        '2015-60 cumulative MSW reduction': [ref1_input4, ref2_input4, diet_input4, waste_input4]
        }

report_df = pd.DataFrame.from_dict(report)
report_df


,Solutions implemented,Scenario comparison,2020-50 cumulative reduction,2015-60 cumulative reduction,2020-50 cumulative MSW reduction,2015-60 cumulative MSW reduction
0,Family Planning,REF1-REF2,4.129998,8.398271,4.129998,8.398271
1,Plant-rich Diet ONLY (75%),REF2-DIET,8.137960,13.790854,8.137960,13.790854
2,Food Waste ONLY (75%),REF2-WASTE,48.513925,78.389217,48.513925,78.389217
3,Integrated Food Solutions (Diet + Waste),REF2-PDS,52.491318,84.114791,52.491318,84.114791
